In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from statistics import mode
import os
import pymysql
import turicreate

In [2]:
# get connection 
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = '12345678'#os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

conn = pymysql.connect(
    host='192.168.0.104',
    port=int(3306),
    user='root',
    passwd = 'root',
    db = "hackathon",
    charset='utf8mb4')

cursor = conn.cursor()
sql = 'insert into modeltables (user_id, notiifcation_type, parent_id, cat_id, item_id, schedule_notification) values(%d,%s,%d,%d,%d,%s)'

In [3]:
category = pd.read_csv('category.csv', index_col=0)
parent1_correlation = pd.read_csv('parent1_corelationMatrix.csv', index_col=0)
parent2_correlation = pd.read_csv('parent2_corelationMatrix.csv', index_col=0)
parent3_correlation = pd.read_csv('parent3_corelationMatrix.csv', index_col=0)
recommended_frequency = pd.read_csv('Recommended_Frequency.csv', index_col=0)
rfm = pd.read_csv('normalized_rfm.csv', index_col=0)
segment_mapping = pd.read_csv('Segment_Mapping.csv', index_col=0)
user_category = pd.read_csv('user_categories.csv', index_col=0)
events = pd.read_csv('events_preprocessed.csv')

In [4]:
categories = segment_mapping.index.tolist()
category_dict = {}
category_dict[categories[0]] = rfm[(rfm.recency >=4) & (rfm.frequency >= 4) & (rfm.monetary >= 4)].visitor_id.values
category_dict[categories[1]] = rfm[(rfm.recency >= 2) & (rfm.frequency >= 3) & (rfm.monetary >= 3)].visitor_id.values
category_dict[categories[2]] = rfm[(rfm.recency >= 3) & (rfm.frequency <= 3) & (rfm.monetary <= 3)].visitor_id.values
category_dict[categories[3]] = rfm[(rfm.recency >= 4) & (rfm.frequency <= 1) & (rfm.monetary <=1)].visitor_id.values
category_dict[categories[4]] = rfm[((rfm.recency >= 4) | (rfm.recency == 3)) & (rfm.frequency <= 1) & (rfm.monetary <=1)].visitor_id.values
category_dict[categories[5]] = rfm[((rfm.recency == 2) | (rfm.recency == 3)) & ((rfm.frequency == 2) | (rfm.frequency == 3)) & ((rfm.monetary == 2) | (rfm.monetary == 3))].visitor_id.values
category_dict[categories[6]] = rfm[((rfm.recency == 2) | (rfm.recency == 3)) & (rfm.frequency <= 2) & (rfm.monetary <=2)].visitor_id.values
category_dict[categories[7]] = rfm[(rfm.recency <= 2) & ((rfm.frequency <= 5) | (rfm.frequency >= 2)) & ((rfm.monetary <=5) | (rfm.monetary >= 2))].visitor_id.values
category_dict[categories[8]] = rfm[(rfm.recency <= 1) & ((rfm.frequency == 4) | (rfm.frequency == 5)) & ((rfm.monetary == 4) | (rfm.monetary == 5))].visitor_id.values
category_dict[categories[9]] = rfm[((rfm.recency == 1) | (rfm.recency == 2)) & ((rfm.frequency == 1) | (rfm.frequency == 2)) & ((rfm.monetary == 1) | (rfm.monetary == 2))].visitor_id.values
category_dict[categories[10]] = rfm[(rfm.recency <= 2) & (rfm.frequency <= 2) & (rfm.monetary <=2)].visitor_id.values

In [5]:
# insert into the database. You need to commit though!
def insert_(cursor, user_id, notification_type, parent_id, cat_id, item_id, schedule_notification) :
    cursor.execute(sql, (user_id, notification_type, parent_id, cat_id, item_id, schedule_notification))

In [6]:
# gets the parent_id of the current item using category table
def getParent(item_id) :
    return category.loc[item_id]['parent_id']

In [7]:
# gets the category id of the current item using category table
def getCategory(item_id) :
    return category.loc[item_id]['category_id']

In [8]:
# gets the Items that is most purchase correlated to the current item using correlation matrix
# def getCorrelatedItem(item_id) :
#     p_id = getParent(item_id)
#     if p_id == 1 :
#         return int(parent1_correlation.loc[item_id].idxmax())
#     elif p_id == 2 :
#         return int(parent2_correlation.loc[item_id].idxmax())
#     else :
#         return int(parent3_correlation.loc[item_id].idxmax())

In [9]:
# returns the next timestamp that will have the recommended hour and recommended weekday
def getClosestTimestamp(recommended_hour, recommended_weekday) :
    current_date = pd.to_datetime(time.time() * 1000, unit='ms')
    hour_difference = (recommended_hour)- (current_date.hour)
    if hour_difference < 0 :
        hour_difference += 24
    day_difference = recommended_weekday - current_date.weekday()
    if day_difference :
        day_difference += 7  
    return int(current_date.timestamp() + hour_difference * 60*60 + day_difference * 86400)


In [ ]:
import turicreate
recommend_ds = events.drop(["Unnamed: 0","timestamp","transactionid"],axis=1)
recommend_ds=recommend_ds.rename(columns = {'visitor_id':'user_id'})
sf = turicreate.SFrame(recommend_ds)
m = turicreate.ranking_factorization_recommender.create(sf)

recommendations = m.recommend()

In [10]:
# called the number of times specified by the recommended_frequency table
def recommend(user_id) :
    user_events = events[events.visitor_id == user_id]

    # hour of the day the next notification should be sent
    send_hour = user_events.hour.mode()[0]

    # weekday on which next notification should be sent
    send_weekday = user_events.weekday.mode()[0]

    # the closest timestamp at which the current criteria is met
    
    # decide the type of notification that the user should be sent

    categories

    # for that we need to know the segments the user is in
    u_categories = []
    #for i in category_dict.keys() :
    [u_categories.append(i) if (user_id in category_dict[i]) else 1==1 for i in category_dict.keys()]
    u_categories

    # now get the possible type of notification that can be sent to the user
    opposite = segment_mapping.transpose()

    possible_notifications = []
    for cat in u_categories:
        possible_notifications = possible_notifications + opposite[opposite[cat] == 1].index.values.tolist() 
    possible_notifications
    
    # get 10 most recent products that the particular user has bought in the past
    item_list = events[(events.visitor_id == user_id) & (events.event == 2)].sort_values(by='timestamp').item_id.tolist()[:10]


    # get item which has highest correlation to the current item from the matrix

    recommendations = m.recommend(users=[user_id])
    x = list(recommendations.select_columns(['item_id']))
    recommended_list = list()
    for item in x:
        for key,value in item.items():
            recommended_list.append(value)
            
    return (send_hour, send_weekday, possible_notifications, recommended_list)


In [32]:
output = recommend(1127)
y = getClosestTimestamp(output[0],output[1])
pd.to_datetime(y * 1000, unit='ms')

Timestamp('2019-03-04 15:26:26')

In [33]:
output

(15,
 0,
 ['Sale Promotions',
  'Newsletters',
  'Customer Testimonials',
  'First-order-only coupons',
  'Order Confirmation',
  'Time-sensitive Coupons',
  'Big Product Update',
  'How-can we help? questionaires',
  'Sale Promotions',
  'Welcome Email Series',
  'Cart Recovery Prompts',
  'Newsletters',
  'Order Confirmation',
  'Educational Content',
  'Educational Content.1',
  'Membership',
  'Company Announcements',
  'Customer Testimonials',
  'Product Recommendation',
  'Membership'],
 [3294, 656, 543, 3298, 4826, 2091, 7633, 3080, 6733, 840])

In [37]:
#insert_(cursor, 9537, mode(x[2]), getParent(x[3][0]), getCategory(x[3][0]), x[3][0], getClosestTimestamp(x[0], x[1]))
sql ="insert into modeltables (user_id, notiifcation_type, parent_id, cat_id, item_id, schedule_notification) values ({0}, '{1}', {2}, {3}, {4}, '{5}');"
sql = sql.format(9537, str(output[2][0]), getParent(output[3][0]), getCategory(output[3][0]),output[3][0], pd.to_datetime(getClosestTimestamp(14,5) * 1000, unit='ms'))
sql

"insert into modeltables (user_id, notiifcation_type, parent_id, cat_id, item_id, schedule_notification) values (9537, 'Sale Promotions', 2, 1, 3294, '2019-03-02 14:30:25');"

In [38]:
cursor.execute(sql)
#df = cursor.execute("insert into modeltables (user_id, notiifcation_type, parent_id, cat_id, item_id, schedule_notification) values (3, 'check', 1, 3, 111, 30001010);")

1

In [39]:
conn.commit()

In [14]:
 = int(time.time() * 1000) + 60 * 3

In [15]:
x

1551532430349

In [23]:
z = pd.to_datetime(time.time() * 1000, unit='ms')

In [29]:
int(z.timestamp()) * 1000

1551532572000